# Initializations

In [1]:
import numpy as np
import pandas as pd

import sys, os, time
import glob

from matplotlib import pyplot as plt
%matplotlib inline

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

# widgets and interaction
from ipywidgets import FloatProgress
from IPython.display import display, clear_output

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

from skimage.io import imread, imsave

import warnings
warnings.filterwarnings('ignore')

import gzip
import cPickle as pickle

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.utils as vutils
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

import sys
sys.path.append("../models/")

import models.dcgan as dcgan
import models.mlp as mlp
import models.dcgan_orig as do

#### Set up batching

In [42]:
import sys
sys.path.append("./../models/pytorch_utils")
from loader_dataframe import ImageDataFrame, grayscale_loader, WeightedRandomSampler

from PIL import Image

def fn_rotate(img):
    theta = np.random.randn()*180
    return img.rotate(theta, expand=True)

pxCrop = 448

dataset = ImageDataFrame(df=train_df, 
                         classCol=classCol,
                         loader=grayscale_loader,
                         transform=transforms.Compose([
                               transforms.RandomHorizontalFlip(),
                               # transforms.RandomCrop(pxCrop),
                               transforms.CenterCrop(pxCrop),
                               transforms.Lambda(lambda img: fn_rotate(img)),
                               transforms.Scale(imageSize),
                               transforms.ToTensor(),
                               # transforms.Normalize((mu, mu, mu), (std, std, std)),
                           ]))
n_classes = len(dataset.classes)
train_df['']
weights = train_df['built pct']

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,
                                         shuffle=True, 
                                         sampler=WeightedRandomSampler(weights, len(train_df)),
                                         num_workers=int(workers))